In [1]:
import numpy as np
import pandas as pd
from glob import glob
import networkx as nx
import os
import re
from tqdm import tqdm
from multiprocess import Pool

In [2]:
tqdm.pandas()

/Users/syeehyn/opt/anaconda3/envs/scipy-ml/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


- A: If appi contains APIj, then aij = 1; otherwise, aij =0.
- B: If APIi and APIj co-exist in the same code block, then bij = 1; otherwise, bij = 0.
- P: If APIi and APIj are with the same package name, then pij = 1; otherwise, pij = 0.
- I: If APIi and APIj use the same invoke method, then iij = 1; otherwise, iij = 0.

- need to change parameter to **cfg

In [86]:
def get_app_smali_dir(fp, app):
    app_dir = fp + '/' + app + '/' + app
    return sorted(glob(os.path.join(app_dir, 'smali*/**/*.smali'), recursive=True))

In [87]:
get_app_smali_dir(fp,'instagram')

['../data/apps/instagram/instagram/smali/X/000.smali',
 '../data/apps/instagram/instagram/smali/X/001.smali',
 '../data/apps/instagram/instagram/smali/X/002.smali',
 '../data/apps/instagram/instagram/smali/X/003.smali',
 '../data/apps/instagram/instagram/smali/X/004.smali',
 '../data/apps/instagram/instagram/smali/X/005.smali',
 '../data/apps/instagram/instagram/smali/X/006.smali',
 '../data/apps/instagram/instagram/smali/X/007.smali',
 '../data/apps/instagram/instagram/smali/X/008.smali',
 '../data/apps/instagram/instagram/smali/X/009.smali',
 '../data/apps/instagram/instagram/smali/X/00A.smali',
 '../data/apps/instagram/instagram/smali/X/00B.smali',
 '../data/apps/instagram/instagram/smali/X/00C.smali',
 '../data/apps/instagram/instagram/smali/X/00D.smali',
 '../data/apps/instagram/instagram/smali/X/00E.smali',
 '../data/apps/instagram/instagram/smali/X/00F.smali',
 '../data/apps/instagram/instagram/smali/X/00G.smali',
 '../data/apps/instagram/instagram/smali/X/00H.smali',
 '../data/

In [88]:
def get_smali_dir(fp, app_list):
    out = {}
    for i in app_list:
        out[i] = get_app_smali_dir(fp, i)
    return out

In [89]:
fp = '../data/apps'
get_smali_dir(fp, ['plague-inc', 'instagram'])

{'plague-inc': ['../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/FastSafeIterableMap.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$1.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$AscendingIterator.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$DescendingIterator.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$Entry.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$IteratorWithAdditions.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$ListIterator.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap$SupportRemove.smali',
  '../data/apps/plague-inc/plague-inc/smali/android/arch/core/internal/SafeIterableMap.smali',
  '../data/apps/plague-inc/plague

In [90]:
def get_method_call(fn):
    with open(fn) as f:
        pattern = r'\.method\s+(.*)|(invoke-.*)'
        data = re.findall(pattern, f.read())
        if len(data) == 0: return pd.DataFrame()
        data = pd.DataFrame(np.array(data), columns = ['method', 'call']).replace('', np.NaN)
        data['method'] = data['method'].fillna(method = 'ffill')
        data['smali'] = [fn.split('/')[-1] for _ in range(len(data))]
    return data.dropna()

In [91]:
fn = '../data/apps/instagram/instagram/smali/androidx/activity/ComponentActivity.smali'
get_method_call(fn)

,method,call,smali
1,public constructor <init>()V,"invoke-direct {p0}, Landroidx/core/app/Compone...",ComponentActivity.smali
2,public constructor <init>()V,"invoke-direct {v0, p0}, LX/0Yh;-><init>(LX/0Yg;)V",ComponentActivity.smali
5,public onCreate(Landroid/os/Bundle;)V,"invoke-static {v0}, LX/0Rl;->A00(I)I",ComponentActivity.smali
6,public onCreate(Landroid/os/Bundle;)V,"invoke-super {p0, p1}, Landroidx/core/app/Comp...",ComponentActivity.smali
7,public onCreate(Landroid/os/Bundle;)V,"invoke-static {p0}, LX/0Yl;->A00(Landroid/app/...",ComponentActivity.smali
8,public onCreate(Landroid/os/Bundle;)V,"invoke-static {v0, v1}, LX/0Rl;->A07(II)V",ComponentActivity.smali
10,public onSaveInstanceState(Landroid/os/Bundle;)V,"invoke-static {v1, v0}, LX/0Yh;->A04(LX/0Yh;LX...",ComponentActivity.smali
11,public onSaveInstanceState(Landroid/os/Bundle;)V,"invoke-super {p0, p1}, Landroidx/core/app/Comp...",ComponentActivity.smali


In [97]:
def get_app_info(fp, app):
    ## Future: using dask to parallel
    agg = [get_method_call(fn) for fn in tqdm(get_app_smali_dir(fp, app))]
    df = pd.concat(agg, ignore_index = True).drop_duplicates()
    pattern =  (
        "(invoke-\w+)(?:\/range)? {.*}, "     # invoke
        + "(\[*[ZBSCFIJD]|\[*L[\w\/$-]+;)->"   # package
        + "([\w$]+|<init>).+"                 # method
    )
    df['block'] = df['method'] + df['smali']
    df = pd.concat([df, df.call.str.extract(pattern)\
            .rename(columns={0: 'invocation', 1: 'package', 2: 'method_name'})], axis = 1)\
            .drop(['call', 'method', 'smali'], axis = 1)\
            .reset_index(drop = True)
    return df

In [98]:
get_app_info(fp, 'plague-inc')

100%|██████████| 7947/7947 [00:19<00:00, 405.64it/s]


,block,invocation,package,method_name
0,public constructor <init>()VFastSafeIterableMa...,invoke-direct,Landroid/arch/core/internal/SafeIterableMap;,<init>
1,public constructor <init>()VFastSafeIterableMa...,invoke-direct,Ljava/util/HashMap;,<init>
2,public ceil(Ljava/lang/Object;)Ljava/util/Map$...,invoke-virtual,Landroid/arch/core/internal/FastSafeIterableMap;,contains
3,public ceil(Ljava/lang/Object;)Ljava/util/Map$...,invoke-virtual,Ljava/util/HashMap;,get
4,public contains(Ljava/lang/Object;)ZFastSafeIt...,invoke-virtual,Ljava/util/HashMap;,containsKey
...,...,...,...,...
123075,public videoViewCleanup()VMraidView.smali,invoke-virtual,Landroid/widget/VideoView;,stopPlayback
123076,public videoViewCleanup()VMraidView.smali,invoke-interface,Landroid/webkit/WebChromeClient$CustomViewCall...,onCustomViewHidden
123077,public videoViewCleanup()VMraidView.smali,invoke-virtual,Ljava/lang/Exception;,printStackTrace
123078,public videoViewCleanup()VMraidView.smali,invoke-virtual,Lcom/tapjoy/mraid/view/MraidView;,setVisibility


In [99]:
def get_apps_info(fp, applist):
    ## Future: using dask/spark to parallel
    out = pd.DataFrame()
    for app in applist:
        df = get_app_info(fp, app)
        df['app'] = [app for _ in range(len(df))]
        out = pd.concat([out, df], ignore_index = True)
    out = out.reset_index(drop = True)
    return out

In [100]:
fp = '../data/apps'
applist = ['plague-inc', 'instagram']
get_apps_info(fp, applist)

100%|██████████| 42619/42619 [01:38<00:00, 432.00it/s]


,block,invocation,package,method_name,app
0,public constructor <init>()VFastSafeIterableMa...,invoke-direct,Landroid/arch/core/internal/SafeIterableMap;,<init>,plague-inc
1,public constructor <init>()VFastSafeIterableMa...,invoke-direct,Ljava/util/HashMap;,<init>,plague-inc
2,public ceil(Ljava/lang/Object;)Ljava/util/Map$...,invoke-virtual,Landroid/arch/core/internal/FastSafeIterableMap;,contains,plague-inc
3,public ceil(Ljava/lang/Object;)Ljava/util/Map$...,invoke-virtual,Ljava/util/HashMap;,get,plague-inc
4,public contains(Ljava/lang/Object;)ZFastSafeIt...,invoke-virtual,Ljava/util/HashMap;,containsKey,plague-inc
...,...,...,...,...,...
703591,private constructor <init>(ILjava/lang/String;...,invoke-direct,Ljava/io/IOException;,<init>,instagram
703592,private constructor <init>(ILjava/lang/String;...,invoke-static,Lpl/droidsonroids/gif/GifError;,fromCode,instagram
703593,public static fromCode(I)Lpl/droidsonroids/gif...,invoke-direct,Lpl/droidsonroids/gif/GifIOException;,<init>,instagram
703594,public getMessage()Ljava/lang/String;GifIOExce...,invoke-virtual,Lpl/droidsonroids/gif/GifError;,getFormattedDescription,instagram


In [101]:
app_info = get_apps_info(fp, applist)

100%|██████████| 42619/42619 [01:37<00:00, 439.00it/s]


In [105]:
app_info

,block,invocation,package,method_name,app
0,public constructor <init>()VFastSafeIterableMa...,invoke-direct,Landroid/arch/core/internal/SafeIterableMap;,<init>,plague-inc
1,public constructor <init>()VFastSafeIterableMa...,invoke-direct,Ljava/util/HashMap;,<init>,plague-inc
2,public ceil(Ljava/lang/Object;)Ljava/util/Map$...,invoke-virtual,Landroid/arch/core/internal/FastSafeIterableMap;,contains,plague-inc
3,public ceil(Ljava/lang/Object;)Ljava/util/Map$...,invoke-virtual,Ljava/util/HashMap;,get,plague-inc
4,public contains(Ljava/lang/Object;)ZFastSafeIt...,invoke-virtual,Ljava/util/HashMap;,containsKey,plague-inc
...,...,...,...,...,...
703591,private constructor <init>(ILjava/lang/String;...,invoke-direct,Ljava/io/IOException;,<init>,instagram
703592,private constructor <init>(ILjava/lang/String;...,invoke-static,Lpl/droidsonroids/gif/GifError;,fromCode,instagram
703593,public static fromCode(I)Lpl/droidsonroids/gif...,invoke-direct,Lpl/droidsonroids/gif/GifIOException;,<init>,instagram
703594,public getMessage()Ljava/lang/String;GifIOExce...,invoke-virtual,Lpl/droidsonroids/gif/GifError;,getFormattedDescription,instagram
